<a href="https://colab.research.google.com/github/hicksteilchen/clone_brain/blob/main/Copy_of_brain_stl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Print your brain 🧠

## Introduction

Cheers 🎉 By stumbling across this page you are only **~1 minute away** from downloading a **3D model of your brain** 🧠

Besides ~1 minute of your time, there are the following **requirements**
- **MRI scan of your head** (Often given to the patient via CD)
  - No MRI scan available? Don't worry, the notebook will automatically use a publicly available MRI 🌐
  - Motivated to get scanned? Google e.g. "neuroimaging study YOUR_CITY" and get scanned (+ help science 🤓)
- One of the following options:
  - **Google account** to run your copy of this Google Colab notebook ☁️
  - **Python installed** on your PC to run the code without Colab 🖥️

## TL;DR

Running a copy of this notebook (*File* -> *Save a copy in Drive*, *Runtime* -> *Run all*) results in a **`brain.stl` file** (folder icon on the left) 📁

This file **can be printed** by e.g. [Shapeways](https://www.shapeways.com/) or your own 3D printer 🖨️

It takes just a **few seconds** to input your own **MRI scan** to get a **3D model of YOUR brain** 🧠

If you like this stuff, give [brain-stl](https://github.com/codingfisch/brain-stl) (+[deepmriprep](https://github.com/wwu-mmll/deepmriprep) and [niftiview](https://github.com/codingfisch/niftiview)) a star on GitHub ⭐

![GIF](https://github.com/user-attachments/assets/64f10b4f-b136-4e86-8ef4-37f9fe0d8fa7)

In [ ]:
# @title Installation
#!apt install libcairo2-dev
!pip install brain-stl

## Run 🏃

To input a MRI scan in your copy of this notebook (File -> Save a copy in Drive)
- Click on the folder icon on the left 📁
- Drag&drop the scan file/folder in there to **upload it** 📤
- Change `scan_filepath`: Replace `None` with the name of the file/folder
- (optional) Enable **faster processing** via *Runtime* -> *Change runtime type* -> *T4 GPU* 🔥
- **Run** via *Runtime* -> *Run all* 🏃

Wait ⏳

- Click on the folder icon on the left (+ click 🔄 to refresh) to look for the **`brain.stl` file** 🧠
- To download `brain.stl`, click on the three dots ⋮ located next to it 🏁

In [ ]:
# @title Run
from brain_stl import run_brain_stl

settings = {'threshold': 1.5,
            'hollow': 0,
            'in_template_space': False,
            'mesh_reduction': 0.0,
            'smooth_iter': 0,
            'smooth_lamb': 0.2,
            'use_cache': False,
            'text': None,
            'text2': None,
            'textsize': 120}

run_brain_stl(scan_filepath=None,  # e.g 'path/to/brain.nii.gz' or 'path/to/brain_folder'
              nifti_name=None, # only used if scan_filepath is folder(=DICOM)
              **settings);

## Extra tips

### `use_cache` 💨

After the first run, `use_cache` can be set to `True` in the **Extra settings** for faster processing

### Customize `brain.stl` 🖌️

Read the [README](https://github.com/codingfisch/brain-stl) (+ask ChatGPT) to customize your brain model via the **settings**

### View `brain.stl` 👀

Use https://f3d.app/web/ to view your `brain.stl`

### Fix DICOM error 🐛

**If you run into the error** `The inputted DICOM...resulted in...Nifti files...`
- Click on the folder icon on the left 📁
- Double-click on `pick_t1w_image.png`
- Follow the instructions to pick the proper `nifti_name` and set it in **Setup**
- **Run** via *Runtime* -> *Run all*

### Create `brain.gif`

To get a GIF of your `brain.stl`
- Remove the # in the last line in the **GIF** code cell
- Run the code cell

**If you wanna show everyone what a brain you are, post your GIF with the hashtag `#brain-stl` on social media** 🤓

In [ ]:
# @title Fix for weird Colab bug
import locale
locale.getpreferredencoding = lambda: 'UTF-8'

In [ ]:
# @title Install pyrender
!pip install pyrender

In [ ]:
# @title GIF
# Modified from https://github.com/codingfisch/brain-stl/blob/main/create_gif.py
import os
import trimesh
import pyrender
import numpy as np
from tqdm import tqdm
from PIL import Image
from pathlib import Path
os.environ['PYOPENGL_PLATFORM'] = 'egl'


def save_brain_gif(stl_filepath, output_folder='.', gif_name='brain', fps=33, quality=90, **kwargs):
    gif_filepath = f'{output_folder}/{gif_name}.gif'
    mesh = trimesh.load_mesh(stl_filepath)
    frames = get_gif_frames(mesh, **kwargs)
    frames[0].save(gif_filepath, append_images=frames[1:], save_all=True, duration=int(1000 / fps), loop=0)


def get_gif_frames(mesh, n=120, size=(480, 360), radius=190, fov=60, light=8, light_angle=(80, 80), bg_color=(0, 0, 0)):
    camera_pose = np.array([[-1, 0, 0, 0], [0, 0, -1, -radius], [0, 1, 0, 0], [0, 0, 0, 1]])
    light_pose = np.array([[1, 0, 0, light_angle[0]], [0, 1, 0, -radius], [0, 0, 1, light_angle[1]], [0, 0, 0, 1]])
    scene = pyrender.Scene(bg_color=bg_color)
    camera = pyrender.PerspectiveCamera(yfov=fov * np.pi / 180)
    light = pyrender.PointLight(color=np.ones(3), intensity=light * 10000)
    renderer = pyrender.OffscreenRenderer(viewport_width=size[0], viewport_height=size[1])
    scene.add(camera, pose=camera_pose)
    scene.add(light, pose=light_pose)
    frames = []
    for angle in tqdm(range(0, 360, 360 // n)):
        pose = trimesh.transformations.rotation_matrix((angle + 180) * np.pi / 180, direction=[0, 0, 1])
        scene.add(pyrender.Mesh.from_trimesh(mesh), pose=pose)
        color, depth = renderer.render(scene)
        frames.append(Image.fromarray(color))
        scene.remove_node(list(scene.mesh_nodes)[0])
    return frames


save_brain_gif('brain.stl')